# ELYZA-tasks-100 による性能評価

In [ ]:
#!pip install accelerate autoawq datasets jsonlines sentencepiece
!pip install accelerate datasets jsonlines sentencepiece
# https://github.com/oobabooga/text-generation-webui/issues/4517#issuecomment-1815279467
!pip install https://github.com/casper-hansen/AutoAWQ/releases/download/v0.1.7/autoawq-0.1.7+cu118-cp310-cp310-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.6/76.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.0/154.0 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.0/229.0 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.7/227.7 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━

## ELYZA-tasks-100 データセットのダウンロード

モデルのダウンロードには 🤗datasets ライブラリを用いる。

データは `/content/dataset.jsonl` に保存する。

In [ ]:
import json
from pathlib import Path

from datasets import load_dataset

# Load the dataset
ds = load_dataset("elyza/ELYZA-tasks-100")

# Function to convert dataset to JSONL format and print
def dataset_to_jsonl(dataset, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        for entry in dataset:
            # Construct JSON object
            json_obj = {
                "input_text": entry['input'],
                "output_text": entry['output'],
                "eval_aspect": entry['eval_aspect']
            }

            # Write JSON object to file in JSONL format
            json_str = json.dumps(json_obj, ensure_ascii=False)
            file.write(json_str + '\n')

# Convert and write the dataset to a file in JSONL format
path_jsonl = Path("/content/dataset.jsonl")

if (path_jsonl.parent is not None) and (not path_jsonl.parent.exists()):
    path_jsonl.parent.mkdir(parents=True, exist_ok=True)

dataset_to_jsonl(ds["test"], path_jsonl)


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating test split: 0 examples [00:00, ? examples/s]

## 評価

ELYZA-tasks-100 データセットを用いて、指定したモデルの評価を実行する。

### ハイパーパラメータについて

ハイパーパラメータは 🤗HuggingFace の Code Snippet のとおりとする（公開者が指定したハイパーパラメータを、チューン済みとみなす）。
ただし、長文の回答を要求する設問もあるため、`max_new_tokens` は `1024` に統一する。

In [ ]:
import json
import jsonlines
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Setup model and tokenizer
model_name = "tokyotech-llm/Swallow-7b-instruct-hf"
tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    device_map="auto")
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    device_map="auto",
)

def format_prompt(input_text):
    # ✋プロンプトは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    prompt_template = (
        "以下に、あるタスクを説明する指示があります。"
        "リクエストを適切に完了するための回答を記述してください。\n\n"
        "### 指示:\n{input}\n\n### 応答:"
    )
    return prompt_template.format(input=input_text)

def generate_text(input_text):
    formatted_prompt = format_prompt(input_text)
    input_ids = tokenizer.encode(
        formatted_prompt,
        add_special_tokens=False,
        return_tensors="pt"
    )

    # Set seed for reproducibility
    seed = 42
    torch.manual_seed(seed)

    # ✋Hparamsは、モデルに合わせて調整（HuggingFace のモデルカード参照）
    tokens = model.generate(
        input_ids.to(device=model.device),
        max_new_tokens=1024,
        temperature=0.99,
        top_p=0.95,
        do_sample=True,
        #pad_token_id=tokenizer.pad_token_id,
        #bos_token_id=tokenizer.bos_token_id,
        #eos_token_id=tokenizer.eos_token_id
    )

    # Remove the input tokens from the generated tokens before decoding
    output_tokens = tokens[0][len(input_ids[0]):]
    return tokenizer.decode(output_tokens, skip_special_tokens=True)

def process_dataset(input_file, output_file):
    # Read dataset and generate predictions
    with jsonlines.open(input_file) as reader, jsonlines.open(output_file, mode='w') as writer:
        for obj in reader:
            prompt = obj['input_text']
            generated_text = generate_text(prompt)
            writer.write({"pred": generated_text})

            print("==============================")
            print(f"Q. {prompt}")
            print(f"A. {generated_text}")
            print()

# Process the dataset
input_dataset = '/content/dataset.jsonl'
output_predictions = '/content/preds.jsonl'
process_dataset(input_dataset, output_predictions)

tokenizer_config.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/914k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/721 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00003.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/3.77G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/203 [00:00<?, ?B/s]

Q. 仕事の熱意を取り戻すためのアイデアを5つ挙げてください。
A. 仕事の熱意を取り戻す5つのアイデアを紹介します。

1.達成したい目標を設定する。

2.同僚と積極的に関わり、支援を求める。

3.タスクの優先順位を明確にする。

4.プロジェクトを小さく分解し、段階的に進める。

5.新しいテクノロジーを学び、ツールの使い方を向上させる。

Q. クマが海辺に行ってアザラシと友達になり、最終的には家に帰るというプロットの短編小説を書いてください。
A. 「クマの物語」。

昔々、あるクマが家を出て、海岸に行ってアザラシに会うことにした。クマは、アザラシと友達になりたかったのですが、誰もがそう思っていたわけではありませんでした。クマが海岸に向かって歩いていると、小さな岩の上にアザラシが乗っているのが見えました。クマはとても嬉しかったですが、自分の存在がアザラシに迷惑をかけるとは考えていませんでした。

アザラシは、クマの存在についての声明を発表した。「私はクマのことが好きだよ」。それを聞いたクマは、それが真実であるかどうか、自分の目で確かめるためにアザラシの近くに行くことに決めた。

アザラシに近づいたクマは、アザラシが声明を発表したことには正当な理由があると感じました。アザラシはクマの存在にまったく気づいておらず、すぐに近づいてきました。そして、クマがアザラシに話しかけたとき、アザラシはクマの耳を傾け、彼の言葉に注意深く耳を傾けました。

クマはアザラシと友達になり、何時間も海岸で過ごし、海辺で散歩やゲームをしました。アザラシはクマの親友であり、常にクマの隣にいることができました。

その日の夕方、クマとアザラシは一緒に家に帰ることにしました。アザラシはクマと一緒に家に帰ることを楽しみにしていて、途中でさまざまな楽しいアクティビティをすることを楽しみにしていました。

クマは自宅に戻り、幸せな日を振り返りました。アザラシはクマの家に来て、2人が一緒に過ごした楽しい日について興奮していました。クマは、アザラシを見つけたときから家に帰るまでの時間を常に思い出していたので、彼らの友情は永遠でした。

Q. 古代ギリシャを学ぶ上で知っておくべきポイントは？
A. 古代ギリシャを知るための5つのポイントを紹介します。
ギリシャ神話」と「哲学」。ギリシャは、人間と自然の